Import Carbonara's Data Tools by running the below cell (_ctrl+enter_)

In [ ]:
# <-- Imports

import CarbonaraDataTools as CDT
import ipywidgets as widgets
from IPython.display import display, clear_output
from glob import glob
import numpy as np

1. View all your processed directories, _ctrl+click_ to select which you'd like in your mixture

In [ ]:
# <-- Press play to view the currently available processed directories

preprocessed_directories = glob('newFitData/*')
examp = ['newFitData/Calmodulin1','newFitData/Calmodulin2']

mixture_SelectMultiple = widgets.SelectMultiple(
    options=preprocessed_directories,
    value=examp,
    #rows=10,
    description='Select which directories to mix:',
    layout={'width': '400px','height':'150px'},
    disabled=False
    )

def SelectMultiple_handler(change):
        global selected_directories
        selected_directories = change.new
        with output:
            clear_output(wait=True)

mixture_SelectMultiple.style = {'description_width': '200px','description_height': '25px'}
selected_directories = examp
# Output widget
output = widgets.Output()
mixture_SelectMultiple.observe(SelectMultiple_handler,names='value')
display(mixture_SelectMultiple)
display(output)

2. Enter a name for this mixture prediction

In [ ]:
# <-- Press play to enter a name for your mixture prediction
mixName_text = widgets.Text(
    value='CalmodulinMix',
    placeholder='Enter a name for your mixture prediction:',
    description='Enter a name for your mixture prediction:',
    disabled=False,
    layout={'width': '500px','height':'50px'},   
)

mixName_text.style = {'description_width': '250px','description_height': '25px'}

def mixName_handler(change):
    global mix_name
    mix_name = change.new
    with output:
        clear_output(wait=True)

mix_name = 'CalmodulinMix'
# Output widget
output = widgets.Output()
mixName_text.observe(mixName_handler,names='value')
display(mixName_text)

3. Enter a unique name for the run with these parameters.

In [ ]:
# <-- Press play to enter a name for your mixture prediction
runName_text = widgets.Text(
    value='fitdata',
    placeholder='Enter a name for your run:',
    description='Enter a name for your run:',
    disabled=False,
    layout={'width': '500px','height':'50px'},   
)

runName_text.style = {'description_width': '250px','description_height': '25px'}

def runName_handler(change):
    global run_name
    run_name = change.new
    with output:
        clear_output(wait=True)

run_name = 'fitdata'
# Output widget
output = widgets.Output()
runName_text.observe(runName_handler,names='value')
display(runName_text)

4. Enter the file location for the SAXS file for the mixture.

In [ ]:
# <-- Press play to enter the mixture SAXs file location

saxsFile_text = widgets.Text(
    value='Examples/Calmodulin/apoCAM_average_average_refined_sx.dat',
    placeholder='Enter your SAXS file location:',
    description='Enter your SAXS file location:',
    disabled=False,
    layout={'width': '550px','height':'50px'},   
)

saxsFile_text.style = {'description_width': '200px','description_height': '25px'}

def saxsFile_handler(change):
    global SAXS_file
    global working_path
    SAXS_file = change.new
    with output:
        clear_output(wait=True)

SAXS_file = 'Examples/Calmodulin/apoCAM_average_average_refined_sx.dat'
# Output widget
output = widgets.Output()
saxsFile_text.observe(saxsFile_handler,names='value')
display(saxsFile_text)

5. Select the range of _q_ you'd like to fit to in this run.

In [ ]:
# <-- Run this cell to enter your desired q range for the fit
working_path = 'newFitData/'+mix_name
CDT.create_mixture(selected_directories,mix_name,SAXS_file)
minq_text = widgets.Text(
    value=str(round(np.genfromtxt(working_path+'/Saxs.dat')[:,0].min(),2)),
    placeholder='Enter your min q:',
    description='Enter your min q:',
    disabled=False,
    layout={'width': '250px','height':'50px'},   
)

minq_text.style = {'description_width': '150px','description_height': '25px'}

def minq_handler(change):
    global minq
    minq = change.new
    with output:
        clear_output(wait=True)

minq = round(np.genfromtxt(working_path+'/Saxs.dat')[:,0].min(),2)
# Output widget
output = widgets.Output()
minq_text.observe(minq_handler,names='value')
display(minq_text)

startq_text = widgets.Text(
    value='0.15',
    placeholder='Enter your max q:',
    description='Enter your max q:',
    disabled=False,
    layout={'width': '250px','height':'50px'},   
)

startq_text.style = {'description_width': '150px','description_height': '25px'}

def startq_handler(change):
    global startq
    startq = change.new
    with output:
        clear_output(wait=True)

startq = '0.15'
# Output widget
output = widgets.Output()
startq_text.observe(startq_handler,names='value')
display(startq_text)

maxq = round(np.genfromtxt(working_path+'/Saxs.dat')[:,0].max(),2)

maxq_text = widgets.Text(
    value=str(round(np.genfromtxt(working_path+'/Saxs.dat')[:,0].max(),2)),
    placeholder='Enter your max q:',
    description='Enter your max q:',
    disabled=False,
    layout={'width': '250px','height':'50px'},   
)

maxq_text.style = {'description_width': '150px','description_height': '25px'}

def maxq_handler(change):
    global maxq
    maxq = change.new
    with output:
        clear_output(wait=True)

# Output widget
output = widgets.Output()
maxq_text.observe(maxq_handler,names='value')
display(maxq_text)

6. Select how many distinct predictions you'd like for this run, and how many fitting steps we should take for each prediction.

In [ ]:
# <-- Run this cell to select the number of runs and how many fitting steps per run

no_runs_text = widgets.Text(
    value='10',
    placeholder='Enter the number of runs:',
    description='Enter the number of runs:',
    disabled=False,
    layout={'width': '350px','height':'50px'},   
)

no_runs_text.style = {'description_width': '250px','description_height': '25px'}

def no_runs_handler(change):
    global no_runs
    no_runs = change.new
    with output:
        clear_output(wait=True)

no_runs = '10'
# Output widget
output = widgets.Output()
no_runs_text.observe(no_runs_handler,names='value')
display(no_runs_text)

fit_steps_text = widgets.Text(
    value='10000',
    placeholder='Enter your number of fitting steps:',
    description='Enter your number of fitting steps:',
    disabled=False,
    layout={'width': '350px','height':'50px'},   
)

fit_steps_text.style = {'description_width': '250px','description_height': '25px'}

def fit_steps_handler(change):
    global fit_steps
    fit_steps = change.new
    with output:
        clear_output(wait=True)

fit_steps = '10000'
# Output widget
output = widgets.Output()
fit_steps_text.observe(fit_steps_handler,names='value')
display(fit_steps_text)

7. Write the _.sh_ file for this run.

In [ ]:
CDT.write_run_sh_file('newFitData/'+mix_name,
                    mix_name,
                    run_name,
                    len(selected_directories),
                    minq,
                    maxq,
                    startq,
                    int(fit_steps),
                    int(no_runs),
                    pairedQ=bool(glob('newFitData/'+mix_name+'/fixedDistanceConstraints*.dat'))
                    )

8. You now need to head to the terminal to set off your run!
* First, navigate to the carbonara directory
```shell
cd {carbonara_directory}
```
* Enter _sh_ followed by the script name printed above and hit enter.
```shell
sh {script_name}
```